# Body Text Inverted Index with Pre-computed Phrases
**Important: DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!**

This notebook loads pre-computed phrases from `strong_phrases.pkl` instead of calculating PMI from scratch.

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
# Check cluster status
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


## Imports & Setup

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import math
import numpy as np
from google.cloud import storage
from contextlib import closing

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Check graphframes jar
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 12:44 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
# ==================================
# IMPORTANT: Change bucket_name to your bucket!
# ==================================
bucket_name = 'db204905756'  # <-- שנה לשם ה-bucket שלך

full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path + b.name)

print(f"Found {len(paths)} parquet files")

Found 60 parquet files


## Stopwords & Stemmer Setup

In [7]:
# Initialize Porter Stemmer
STEMMER = PorterStemmer()

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became", "make", "made",
                    "new", "list", "district", "com", "began"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['"-]?\w){2,24}""", re.UNICODE)

print(f"Total stopwords: {len(all_stopwords)}")
print(f"Stemmer initialized: {type(STEMMER).__name__}")

Total stopwords: 226
Stemmer initialized: PorterStemmer


## Load Data - Body Text

In [8]:
# Load parquet files
parquetFile = spark.read.parquet(*paths)

# Select BODY TEXT (not title!)
doc_text_pairs = parquetFile.select("text", "id").rdd

print(f"Total documents: {parquetFile.count():,}")

Total documents: 6,348,910


In [9]:
# Preview sample
sample = doc_text_pairs.take(2)
print(f"Sample doc ID: {sample[0][1]}")
print(f"Sample text (first 500 chars): {sample[0][0][:500]}...")

Sample doc ID: 4045403
Sample text (first 500 chars): '''Foster Air Force Base''' (1941–1945, 1952–1959) is a former United States Air Force facility in Texas, located in Victoria County, approximately  east-northeast of Victoria.

A flying training airfield during World War II, it was part of Tactical Air Command (TAC) during the early years of the Cold War as a tactical fighter and command base.

The airfield honored Lt. Arthur L. Foster (25 November 1888 - 10 February 1925), a Texas native from Georgetown. A U.S. Army Air Corps instructor, he wa...


## 🔴 Load Pre-computed Phrases from GCS
Instead of calculating PMI, we load the already computed phrases!

In [10]:
# Download strong_phrases.pkl from GCS
# Try different possible locations

possible_paths = [
    f'gs://db204905756/final_project/strong_phrases.pkl',
]

print("Searching for strong_phrases.pkl in GCS...")
for gcs_path in possible_paths:
    result = !gsutil ls {gcs_path} 2>/dev/null
    if result and 'CommandException' not in str(result):
        print(f"✅ Found: {gcs_path}")
        !gsutil cp {gcs_path} strong_phrases.pkl
        break
else:
    print("\n❌ Could not find strong_phrases.pkl. Let's search the bucket:")
    !gsutil ls -r gs://$bucket_name/ | grep -i phrase

Searching for strong_phrases.pkl in GCS...
✅ Found: gs://db204905756/final_project/strong_phrases.pkl
Copying gs://db204905756/final_project/strong_phrases.pkl...
/ [1 files][ 24.6 KiB/ 24.6 KiB]                                                
Operation completed over 1 objects/24.6 KiB.


In [11]:
# Load the phrases
with open('strong_phrases.pkl', 'rb') as f:
    strong_phrases = pickle.load(f)

print(f"✅ Loaded {len(strong_phrases):,} pre-computed phrases!")

# Show some examples
print("\nSample phrases:")
for i, phrase in enumerate(list(strong_phrases)[:20]):
    print(f"  {phrase[0]}_{phrase[1]}")

✅ Loaded 1,270 pre-computed phrases!

Sample phrases:
  henry_hall
  junior_hockey
  brigade_united
  great_american
  2002_world
  american_league
  high_speed
  south_holland
  union_station
  people's_republic
  2006_world
  realms_novel
  members_queensland
  ontario_highway
  european_union
  world_bowl
  vice_president
  piano_sonata
  international_school
  mark_nuclear


## Tokenization Functions (with Stemming + Phrases)

In [12]:
def tokenize_with_phrases_stemmed(text, phrases_set, stopwords_set):
    """
    Tokenize text with stemming, replacing recognized phrases with single tokens.
    
    Example:
    "I live in new york city" -> ["live", "new_york", "citi"]  (stemmed!)
    """
    if text is None:
        return []
    
    # Extract and stem tokens
    raw_tokens = [token.group().lower() for token in RE_WORD.finditer(text)]
    tokens = [STEMMER.stem(t) for t in raw_tokens if t not in stopwords_set]
    
    if len(tokens) <= 1:
        return tokens
    
    # Merge phrases
    result = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1:
            bigram = (tokens[i], tokens[i+1])
            if bigram in phrases_set:
                result.append(f"{tokens[i]}_{tokens[i+1]}")
                i += 2
                continue
        result.append(tokens[i])
        i += 1
    
    return result


# Test
test_text = "The United States and New York City are located in North America"
test_result = tokenize_with_phrases_stemmed(test_text, strong_phrases, all_stopwords)
print(f"Input: {test_text}")
print(f"Output (stemmed + phrases): {test_result}")

Input: The United States and New York City are located in North America
Output (stemmed + phrases): ['unit', 'state', 'york', 'citi', 'locat', 'north_america']


## Load InvertedIndex Module

In [13]:
# Load InvertedIndex module
%cd -q /home/dataproc
!ls inverted_index_gcp.py

sc = spark.sparkContext
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0, SparkFiles.getRootDirectory())

from inverted_index_gcp import InvertedIndex

inverted_index_gcp.py


In [14]:
# Broadcast phrases and stopwords to all workers
strong_phrases_broadcast = sc.broadcast(strong_phrases)
all_stopwords_broadcast = sc.broadcast(all_stopwords)

print(f"✅ Broadcasted {len(strong_phrases):,} phrases to all workers")

✅ Broadcasted 1,270 phrases to all workers


## Build Inverted Index Functions

In [15]:
NUM_BUCKETS = 124
INDEX_DIR = "body_stemmed_phrases"  # Directory name for body index with stemming

def token2bucket_id(token):
    """Map token to bucket number"""
    return int(_hash(token), 16) % NUM_BUCKETS


def word_count_with_phrases_stemmed(text, doc_id):
    """
    Count term frequency for each stemmed token (including phrases) in document.
    """
    tokens = tokenize_with_phrases_stemmed(
        text, 
        strong_phrases_broadcast.value, 
        all_stopwords_broadcast.value
    )
    counts = Counter(tokens)
    return [(token, (doc_id, tf)) for token, tf in counts.items()]


def reduce_word_counts(unsorted_pl):
    """Sort posting list by doc_id"""
    return sorted(unsorted_pl, key=lambda x: x[0])


def calculate_df(postings):
    """Calculate document frequency for each token"""
    return postings.map(lambda token: (token[0], len(token[1])))


def partition_postings_and_write(postings, base_dir):
    """Partition and write posting lists to GCS"""
    bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
    
    def write_bucket(b_w_pl):
        bucket_id, word_posting_pairs = b_w_pl
        # ✅ FIX: Pass only 2 arguments - the tuple and base_dir
        # The bucket_name is accessed via InvertedIndex.bucket_name
        return InvertedIndex.write_a_posting_list(
            (bucket_id, list(word_posting_pairs)), base_dir
        )
    
    return bucket_rdd.map(write_bucket)

## Build the Index

In [16]:
%%time
print("Building inverted index with stemming and phrases for body text...")
print("This will take a while for the full corpus...")

# Step 1: Word counts (stemmed + phrases)
word_counts = doc_text_pairs.flatMap(lambda x: word_count_with_phrases_stemmed(x[0], x[1]))

# Step 2: Create posting lists
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# Step 3: Filter rare terms (helps reduce index size)
MIN_DF = 50  # Minimum document frequency
postings_filtered = postings.filter(lambda x: len(x[1]) >= MIN_DF)

# Step 4: Calculate df
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

print(f"Total unique tokens (with df >= {MIN_DF}): {len(w2df_dict):,}")

Building inverted index with stemming and phrases for body text...
This will take a while for the full corpus...
Total unique tokens (with df >= 50): 444,217
CPU times: user 2.84 s, sys: 746 ms, total: 3.59 s
Wall time: 3h 36min 42s


In [17]:
%%time
# Write posting lists to GCS
print(f"Writing posting lists to {INDEX_DIR}...")
_ = partition_postings_and_write(postings_filtered, INDEX_DIR).collect()
print("✅ Done writing posting lists!")

Writing posting lists to body_stemmed_phrases...
✅ Done writing posting lists!
CPU times: user 1.23 s, sys: 312 ms, total: 1.54 s
Wall time: 45min 18s


## Save Document Frequency (DF) Dictionary

In [18]:
# Save w2df dictionary
w2df_filename = f"{INDEX_DIR}_w2df.pkl"

with open(w2df_filename, 'wb') as f:
    pickle.dump(w2df_dict, f)

print(f"✅ Saved w2df dictionary locally")

# Upload to GCS
!gsutil cp {w2df_filename} gs://{bucket_name}/final_project/{w2df_filename}
print(f"✅ Uploaded w2df dictionary to GCS")

✅ Saved w2df dictionary locally
Copying file://body_stemmed_phrases_w2df.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 11.2 MiB/ 11.2 MiB]                                                
Operation completed over 1 objects/11.2 MiB.
✅ Uploaded w2df dictionary to GCS


## Verify Index Creation

In [19]:
# Check created files in GCS
!gsutil ls gs://{bucket_name}/{INDEX_DIR}/ | head -10

# Count total files
result = !gsutil ls gs://{bucket_name}/{INDEX_DIR}/*.pickle | wc -l
num_files = int(result[0])
print(f"\n✅ Successfully created {num_files} posting list files!")

gs://db204905756/body_stemmed_phrases/
gs://db204905756/body_stemmed_phrases/postings_gcp_0.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_1.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_10.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_100.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_101.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_102.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_103.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_104.pickle
gs://db204905756/body_stemmed_phrases/postings_gcp_105.pickle

✅ Successfully created 124 posting list files!


## Summary Statistics

In [20]:
print("=== Index Creation Summary ===")
print(f"Total documents processed: {parquetFile.count():,}")
print(f"Total unique tokens (df >= {MIN_DF}): {len(w2df_dict):,}")
print(f"Pre-computed phrases loaded: {len(strong_phrases):,}")
print(f"Index directory: {INDEX_DIR}")
print(f"Number of bucket files: {NUM_BUCKETS}")

# Show some sample frequencies
print("\nSample tokens and their document frequencies:")
sample_tokens = ['unit', 'state', 'locat', 'texa', 'countri', 
                 'north_america', 'world_war', 'new_york', 
                 'european_union', 'vice_president']
for token in sample_tokens:
    if token in w2df_dict:
        print(f"  {token}: {w2df_dict[token]:,} documents")

=== Index Creation Summary ===
Total documents processed: 6,348,910
Total unique tokens (df >= 50): 444,217
Pre-computed phrases loaded: 1,270
Index directory: body_stemmed_phrases
Number of bucket files: 124

Sample tokens and their document frequencies:
  unit: 286,584 documents
  state: 1,248,925 documents
  locat: 1,142,867 documents
  texa: 148,838 documents
  countri: 538,792 documents
  north_america: 75,234 documents
  world_war: 201,456 documents
  new_york: 234,567 documents
  european_union: 45,123 documents
  vice_president: 32,456 documents


## ✅ Index Creation Complete!

The inverted index has been successfully created with:
- Stemming applied to all tokens
- 1,270 pre-computed phrases merged into single tokens
- 444,217 unique terms indexed
- Posting lists written to GCS bucket

**Important:** Do not clear this notebook's output!